In [1]:
# GPT with covernet input

import importlib
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import pickle
import numpy as np

from nuscenes.prediction.models.backbone import ResNetBackbone

import utilsHannes as utilsH
from utilsHannes import CoverNetNoRelu
from utilsHannes import ConstantLatticeLoss
from utilsHannes import mean_pointwise_l2_distance
importlib.reload(utilsH)

version = "v1.0-mini" # v1.0-mini, v1.0-trainval
DATAROOT = "data/sets/nuscenes"
subset = "mini_train" # 'mini_train', 'mini_val', 'train', 'val'.
seconds_of_history_used = 2.0

#################################################################################################################################
# Define your custom dataset class that inherits from torch.utils.data.Dataset
class NuscenesDataset(Dataset):
    def __init__(self, image_data, agent_state_data, ground_truth_data):
        self.image_data = image_data
        self.agent_state_data = agent_state_data
        self.ground_truth_data = ground_truth_data
        
    def __len__(self):
        return len(self.image_data)
    
    def __getitem__(self, index):
        image_data_item = self.image_data[index]
        agent_state_data_item = self.agent_state_data[index]
        ground_truth_data_item = self.ground_truth_data[index]
        
        return image_data_item, agent_state_data_item, ground_truth_data_item

#################################################################################################################################


In [2]:
# Use get_and_format_data as a wrapping function:
img_list, img_tensor_list, agent_state_vector_list, future_xy_local_list = utilsH.get_and_format_data(version, DATAROOT, subset, seconds_of_history_used)


Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.518 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


/opt/conda/envs/nuscenesDev/lib/python3.8/site-packages/nuscenes/map_expansion/map_api.py:1823: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  exteriors = [int_coords(poly.exterior.coords) for poly in polygons]
/opt/conda/envs/nuscenesDev/lib/python3.8/site-packages/nuscenes/map_expansion/map_api.py:1824: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  interiors = [int_coords(pi.coords) for poly in polygons for pi in poly.interiors]


In [3]:
# For testing
short_size = 100
short_img_tensor_list = img_tensor_list[:short_size]
short_agent_state_vector_list = agent_state_vector_list[:short_size]
short_future_xy_local_list = future_xy_local_list[:short_size]

# Prints
num_datapoints = len(img_tensor_list)
print(f"num_datapoints in whole dataset = {num_datapoints}")
num_datapoints = len(short_img_tensor_list)
print(f"short_num_datapoints = {num_datapoints}")
print(f"img_tensor_list size = {img_tensor_list[0].size()}")
print(f"agent_state_vector_list = {agent_state_vector_list[0]}")
print(f"future_xy_local_list = {future_xy_local_list[0][0]}")


# Variables
batch_size = 10
shuffle = True # Set to True if you want to shuffle the data in the dataloader
num_modes = 64 # 2206, 415, 64 (match with eps_traj_set)
eps_traj_set = 8 # 2, 4, 8 (match with num_modes)
learning_rate = 1e-4 # From Covernet paper: fixed learning rate of 1e−4
num_epochs = 100 

# Define your training dataset
dataset = NuscenesDataset(img_tensor_list, agent_state_vector_list, future_xy_local_list)
shortDataset = NuscenesDataset(short_img_tensor_list, short_agent_state_vector_list, short_future_xy_local_list)

# Instantiate your training dataloader
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
shortDataloader = DataLoader(shortDataset, batch_size=batch_size, shuffle=shuffle)

# Initialize the CoverNet model
backbone = ResNetBackbone('resnet50') 
covernet = CoverNetNoRelu(backbone, num_modes)

# Lattice and similarity function
with open(f'data/sets/nuscenes-prediction-challenge-trajectory-sets/epsilon_{eps_traj_set}.pkl', 'rb') as f:
    lattice = np.array(pickle.load(f))
similarity_function = mean_pointwise_l2_distance


# Define your loss function and optimizer
loss_function = ConstantLatticeLoss(lattice, similarity_function)
optimizer = optim.Adam(covernet.parameters(), lr=learning_rate) 

# Move the model to GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
covernet.to(device)

# Training loop
for epoch in range(num_epochs):
    epochLoss = 0
    for batchCount, batch in enumerate(shortDataloader):

        # Get batch data
        image_tensor, agent_state_vector, ground_truth_trajectory = batch
        
        # Squeeze for correct dimensions
        image_tensor = torch.squeeze(image_tensor, dim=1)
        agent_state_vector = torch.squeeze(agent_state_vector, dim=1)

        # Send to device
        image_tensor = image_tensor.to(device)
        agent_state_vector = agent_state_vector.to(device)
        ground_truth_trajectory = ground_truth_trajectory.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        logits = covernet(image_tensor, agent_state_vector)

        # Compute loss
        loss = loss_function(logits, ground_truth_trajectory)
        epochLoss += loss.item()

        # Backward pass
        loss.backward()
        optimizer.step()

        # Print loss for this batch
        # print(f"Batch [{batchCount+1}/{int(num_datapoints/batch_size)+1}], Batch Loss: {loss.item():.4f}")
     
    # Print loss for this batch
    print(f"Epoch [{epoch+1}/{num_epochs}], Epoch Loss: {epochLoss:.4f}")


    # Optionally, you can evaluate the model after each epoch
    # by running inference on a validation set and computing relevant metrics

# Training complete
print("Training complete!")


num_datapoints in whole dataset = 49
short_num_datapoints = 49
img_tensor_list size = torch.Size([1, 3, 500, 500])
agent_state_vector_list = tensor([[2.7443, 1.5398, 0.0000]])
future_xy_local_list = [-0.05469174  1.92462069]
Epoch [1/100], Epoch Loss: 22.2111
Epoch [2/100], Epoch Loss: 15.4978
Epoch [3/100], Epoch Loss: 15.2080
Epoch [4/100], Epoch Loss: 14.6813
Epoch [5/100], Epoch Loss: 13.0910
Epoch [6/100], Epoch Loss: 11.9241
Epoch [7/100], Epoch Loss: 11.3058
Epoch [8/100], Epoch Loss: 11.6602
Epoch [9/100], Epoch Loss: 10.0338
Epoch [10/100], Epoch Loss: 10.7664
Epoch [11/100], Epoch Loss: 9.0260
Epoch [12/100], Epoch Loss: 9.6350
Epoch [13/100], Epoch Loss: 8.5411
Epoch [14/100], Epoch Loss: 7.7895
Epoch [15/100], Epoch Loss: 7.2916
Epoch [16/100], Epoch Loss: 7.2836
Epoch [17/100], Epoch Loss: 6.0003
Epoch [18/100], Epoch Loss: 5.1073
Epoch [19/100], Epoch Loss: 5.0320
Epoch [20/100], Epoch Loss: 4.9066
Epoch [21/100], Epoch Loss: 4.2622
Epoch [22/100], Epoch Loss: 3.9776
Epoc

KeyboardInterrupt: 